#### Import

In [1]:
import easyocr
import os
import torch
import matplotlib.pyplot as plt
import random
import glob
from tqdm import tqdm
import numpy as np
import time
from mmocr_eval import eval_ocr_metric

#### Install packges

In [2]:
#!pip install matplotlib
#!pip install mmocr

In [3]:
torch.cuda.is_available()

True

#### Training

Load model by easyOCR API (https://www.jaided.ai/easyocr/documentation/)

In [4]:
start = time.time()
reader = easyocr.Reader(lang_list = ['en','th'],
                        model_storage_directory = os.path.join('my_model','model'),
                        user_network_directory = os.path.join('my_model','user_network'),
                        recog_network  ='crnn_30k_2000',
                        gpu = True)
# reader = easyocr.Reader(lang_list = ['en','th'],gpu = True )
end = time.time()
print(f'loading model time: {end - start}')

train_module loaded
my_model\user_network\crnn_30k_2000.yaml
Transformation: None
FeatureExtraction: VGG
SequenceModeling: BiLSTM
Prediction: CTC
loading model time: 4.876008987426758


visualise testing with some image

In [ ]:
plt.rcParams['font.family'] = 'TH Sarabun New'
plt.rcParams['xtick.labelsize'] = 20.0
plt.rcParams['ytick.labelsize'] = 20.0
columns = 4
rows = 4
fig = plt.figure(figsize=(15, 15))
ax = []
for i in range(columns*rows):
    random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]
    random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)
    #random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]
    # random_file_name = os.listdir(os.path.join('trainer/all_data/testing/testing'))[2]
    # random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)
    img = plt.imread(random_file_path)
    result = reader.readtext(random_file_path) 
    ax.append(fig.add_subplot(rows, columns, i+1) )
    if len(result) > 0:
        txt = ''
        for i in result:
            txt += (' '+i[1])
        ax[-1].set_title(f'predict: {txt}',fontsize=20)
    else:
        ax[-1].set_title('No result',fontsize=20)
    plt.imshow(img)
fig.tight_layout() 
plt.show()

testing with all files in dataset

In [6]:
# filenames = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
# label_path = 'trainer/all_data/testing/testing/label.txt'
# with open(label_path, encoding='utf8') as file:
#     label = np.loadtxt(file,dtype=str)
# word_correct = 0
# filenames = sorted(filenames, key= lambda x: int(x.split('/')[-1].split('\\')[1].split('.')[0]))
# start = time.time()
# # filenames = filenames[:10]
# pred = []
# gt = [] # ground truth
# for i in tqdm(range(len(filenames))):
#     label_img = label[i][1]

#     result = reader.readtext(filenames[i], batch_size = 8)
#     try:
#         pred.append(result[0][1])
#         gt.append(label_img)
#     except:
#         pred.append('')
#         gt.append(label_img)
# end = time.time()
# output = eval_ocr_metric(pred, gt)
# for k,v in output.items():
#     print(f'{k}: {v}')
# print(f'time: {end - start}')

testing with LMWN

In [ ]:
# plt.rcParams['font.family'] = 'TH Sarabun New'
# plt.rcParams['xtick.labelsize'] = 20.0
# plt.rcParams['ytick.labelsize'] = 20.0
# columns = 1
# rows = 5
# fig = plt.figure(figsize=(10, 10))
# ax = []
# for i in range(columns*rows):
#     random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/LMWN')),1)[0]
#     # random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)
#     #random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]
#     # random_file_name = os.listdir(os.path.join('trainer/all_data/testing/LMWN'))[2]
#     random_file_path = os.path.join('trainer/all_data/testing/LMWN',random_file_name)
#     img = plt.imread(random_file_path)
#     result = reader.readtext(random_file_path) 
#     ax.append(fig.add_subplot(rows, columns, i+1) )
#     txt = ''
#     if len(result) > 0:
#         for i in result:
#             txt += (' '+i[1])
#         ax[-1].set_title(f'predict: {txt}',fontsize=20)
#     else:
#         ax[-1].set_title('No result',fontsize=20)
#     plt.imshow(img)
# fig.tight_layout() 
# plt.show()